In [109]:
import logging

import numpy as np
import pandas as pd
from astropy.table import Table, join
from astroquery.mast import Catalogs
from plato.stars import gaia
from plato.utils import get_abspath

logging.getLogger("astroquery").setLevel(logging.WARNING)

In [110]:
# load Tess Input Catalog (TIC) ids for Planet Candidates from TESS Targets of Interest list
tic_ids = Table.read(get_abspath() + "data/external/TESS_TOI.csv", comment="#")[
    "tid"
].tolist()

# get Gaia DR2 IDs from TIC using astroquery and match to TIC
mast_catalog = Catalogs.query_criteria(
    catalog="TIC",
    ID=tic_ids,
)
tois = (
    mast_catalog.to_pandas()[["ID", "GAIA"]]
    .dropna()
    .astype(int)
    .rename(columns={"ID": "TIC_ID", "GAIA": "dr2_source_id"})
)

In [128]:
# query the dr2_d3_crossmatch table to get the dr3 ids for the dr2 targets
def dr2_dr3_crossmatch_query(id_list):
    query = f"""
    SELECT
        dr2_source_id,
        dr3_source_id,
        angular_distance,
        magnitude_difference
    FROM
        gaiadr3.dr2_neighbourhood
    WHERE
        dr2_source_id IN ({id_list})
  """
    return query


crossmatch_catalogue = gaia.query(
    dr2_dr3_crossmatch_query, list(tois["dr2_source_id"])
).to_pandas()

# select the closest match (in terms of angular distance) for each toi
crossmatch_catalogue = crossmatch_catalogue.sort_values(
    by=["dr2_source_id", "angular_distance", "magnitude_difference"]
)
crossmatch_catalogue = crossmatch_catalogue.drop_duplicates(
    subset="dr2_source_id", keep="first"
).rename(columns={"dr3_source_id": "SOURCE_ID"})

# add TIC ID back in
crossmatch_catalogue = crossmatch_catalogue.merge(tois, on="dr2_source_id", how="left")

# turn dataframe into back into astropy table
crossmatch_catalogue = Table.from_pandas(
    crossmatch_catalogue[["TIC_ID", "dr2_source_id", "SOURCE_ID"]]
)

100%|██████████| 1/1 [00:06<00:00,  6.83s/it]


In [129]:
# create query to get relevant columns from Gaia DR3 gaia_source and astrophysical_parameters tables
def data_query(id_list):
    query = f"""
    SELECT
        gaiadr3.gaia_source.source_id,
        gaiadr3.gaia_source.parallax,
        gaiadr3.gaia_source.parallax_error,
        gaiadr3.gaia_source.ra,
        gaiadr3.gaia_source.ra_error,
        gaiadr3.gaia_source.dec,
        gaiadr3.gaia_source.dec_error,
        gaiadr3.gaia_source.pmra,
        gaiadr3.gaia_source.pmra_error,
        gaiadr3.gaia_source.pmdec,
        gaiadr3.gaia_source.pmdec_error,
        gaiadr3.gaia_source.radial_velocity,
        gaiadr3.gaia_source.radial_velocity_error,
        astrophysical_parameters.mh_gspphot,
        astrophysical_parameters.mh_gspphot_lower,
        astrophysical_parameters.mh_gspphot_upper,
        astrophysical_parameters.mh_gspspec,
        astrophysical_parameters.mh_gspspec_lower,
        astrophysical_parameters.mh_gspspec_upper,
        astrophysical_parameters.alphafe_gspspec,
        astrophysical_parameters.alphafe_gspspec_lower,
        astrophysical_parameters.alphafe_gspspec_upper,
        astrophysical_parameters.logg_gspphot,
        astrophysical_parameters.logg_gspphot_lower,
        astrophysical_parameters.logg_gspphot_upper,
        astrophysical_parameters.logg_gspspec,
        astrophysical_parameters.logg_gspspec_lower,
        astrophysical_parameters.logg_gspspec_upper,
        astrophysical_parameters.flags_gspspec
    FROM
        gaiadr3.gaia_source
    LEFT JOIN
        gaiadr3.astrophysical_parameters
    ON
        gaiadr3.gaia_source.source_id = gaiadr3.astrophysical_parameters.source_id
    WHERE
        gaiadr3.gaia_source.source_id IN ({id_list});
    """
    return query


targets = gaia.query(data_query, crossmatch_catalogue["SOURCE_ID"].tolist())  # type: ignore

100%|██████████| 1/1 [00:29<00:00, 29.49s/it]


In [130]:
# add DR 2 source IDs back to the table
targets = join(targets, crossmatch_catalogue, keys="SOURCE_ID", join_type="left")

In [131]:
targets["flags_gspspec"] = targets["flags_gspspec"].astype(str)

In [132]:
targets

SOURCE_ID,parallax,parallax_error,ra,ra_error,dec,dec_error,pmra,pmra_error,pmdec,pmdec_error,radial_velocity,radial_velocity_error,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,mh_gspspec,mh_gspspec_lower,mh_gspspec_upper,alphafe_gspspec,alphafe_gspspec_lower,alphafe_gspspec_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,logg_gspspec,logg_gspspec_lower,logg_gspspec_upper,flags_gspspec,TIC_ID,dr2_source_id
,mas,mas,deg,mas,deg,mas,mas / yr,mas / yr,mas / yr,mas / yr,km / s,km / s,dex,dex,dex,dex,dex,dex,dex,dex,dex,log(cm.s**-2),log(cm.s**-2),log(cm.s**-2),log(cm.s**-2),log(cm.s**-2),log(cm.s**-2),,,
int64,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,str41,int64,int64
16870631694208,2.101317240027476,0.017278953,45.11277842681158,0.01631012,0.3808400749444696,0.011937999,-0.9694212744716542,0.017738635,-12.268805117194699,0.014436127,1.8293474,0.47387925,-0.0973,-0.0995,-0.0943,-0.02,-0.07,0.04,0.23,0.18,0.31,3.6465,3.6426,3.6498,4.0,3.82,4.08,00000010000009999999900999900999999999999,347553438,16870631694208
5642251292706560,5.630600975252042,0.111837156,41.0365552377074,0.10401954,3.9102297503213577,0.11184234,35.19180557343749,0.13307947,-13.430418676284175,0.12937282,47.388107,2.2437737,--,--,--,0.06,-0.03,0.13,0.03,-0.02,0.09,--,--,--,3.95,3.83,4.08,00000010000009999999999999999999999999999,318812447,5642251292706560
5900705244680192,11.04055558204551,0.018207239,41.380427222993305,0.01574396,5.150722425576574,0.013853303,169.71209868691895,0.022668412,-76.79166582354438,0.01825901,8.675014,0.92564434,0.3409,0.2962,0.4454,0.13,0.09,0.17,0.14,0.11,0.18,4.5965,4.5914,4.6224,4.66,4.26,4.88,00000010000009999999999999999999999999999,318836983,5900705244680192
6612604664008320,1.5767023050831437,0.014387907,41.50668018717253,0.012912129,6.461722058463088,0.010550172,4.107719899683058,0.017471151,-12.31770112071962,0.014407193,13.485204,4.1824627,-0.7633,-0.7824,-0.7455,--,--,--,--,--,--,4.2818,4.2757,4.2877,--,--,--,,387401076,6612604664008320
8727137322879488,3.144342761407983,0.017588122,44.24507555923245,0.014456175,7.85933013019336,0.016032454,-11.10579860196064,0.019388253,-10.701761158739602,0.020669417,-57.644592,1.518648,0.0934,0.0828,0.104,--,--,--,--,--,--,4.4562,4.4517,4.4662,--,--,--,,387544178,8727137322879488
13173974662864256,6.637852347883698,0.02064897,53.90715031526838,0.01846083,10.842912087774215,0.014394319,-4.896974009982596,0.023862822,-7.328017251033016,0.018664196,-34.099274,0.6833216,-0.1335,-0.1748,-0.119,-0.4,-0.54,-0.22,0.29,0.17,0.38,4.5758,4.564,4.589,5.01,4.89,5.04,00000021000009999999999999999999999999999,416739490,13173974662864256
16781094356187264,3.866890626055434,0.01975974,51.99978411442126,0.017176475,12.52341082053971,0.014079683,-2.977914380051434,0.023701936,-27.508216157417856,0.018300397,-20.893532,3.542653,-0.0775,-0.0929,-0.0599,--,--,--,--,--,--,4.5074,4.5035,4.5113,--,--,--,,435259601,16781094356187264
17487702375676544,4.320601376584021,0.018078152,51.029238909720746,0.01656576,12.43596062334673,0.012866501,23.386021013529234,0.023527598,-20.191654839512108,0.01681375,9.260306,1.7231047,-0.3593,-0.3797,-0.3401,--,--,--,--,--,--,4.433,4.4281,4.4375,--,--,--,,303502327,17487702375676544


In [134]:
# save
targets.write(
    get_abspath() + "data/WIP/TESS_targets.fits", format="fits", overwrite=True
)